In [1]:
# Dependencies
import pymongo
import datetime
import pandas as pd
import re

In [2]:
# Connect to database
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client['carsguide_used_cars_australia']
collection = db.scraped_cars

# Create df
df = pd.DataFrame(list(collection.find()))
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,616017a57591e422f04df82e,Parkside Motor Co,2006 Mercedes-Benz C200 Kompressor,189474,Automatic,8990,Just arrived stunning C200 Kompressor Sport in...,https://www.carsguide.com.au/buy-a-car/all-new...,2006,Mercedes-Benz,"[C200, Kompressor, ]",WA,Maylands,USED Dealer ad,2021/10/08,0
1,616017a57591e422f04df82f,Great Deal Auto,2009 Ford Ranger XL,195565,Automatic,15500,We offer onsite finance through Australias lea...,https://www.carsguide.com.au/buy-a-car/all-new...,2009,Ford,"[Ranger, XL]",WA,Malaga,USED Dealer ad,2021/10/08,0
2,616017a57591e422f04df830,Fabcar,2004 Toyota Estima Aeras,84973,Automatic,29888,** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY ...,https://www.carsguide.com.au/buy-a-car/all-new...,2004,Toyota,"[Estima, Aeras]",WA,Perth,USED Dealer ad,2021/10/08,0
3,616017a57591e422f04df831,Osborne Park Nissan used,2018 Dodge N/A Laramie Crew Cab SWB,106000,Automatic,149999,THOUSANDS SPEND ON UPGRADES!! heavy duty rear ...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,Dodge,"[N/A, Laramie, Crew, Cab, SWB]",WA,Osborne Park,USED Dealer ad,2021/10/08,0
4,616017a57591e422f04df832,Subaru Wangara used,2015 Volkswagen Golf R,102001,Manual,39998,LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefull...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Volkswagen,"[Golf, R]",WA,Wangara,USED Dealer ad,2021/10/08,0


In [3]:
# Clean Data
df = df[df.Price != 'POA']
df['Price'] = df.loc[:,"Price"].astype(float)
df = df[df.Price > 500]
df = df[df.Price < 900000]
df = df[df.Price != 1234]
df = df[df['Dealer Name'] != 'POA']
df = df[df['State Abrev'] != 'Hyundai']
df.head()

,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,616017a57591e422f04df82e,Parkside Motor Co,2006 Mercedes-Benz C200 Kompressor,189474,Automatic,8990.0,Just arrived stunning C200 Kompressor Sport in...,https://www.carsguide.com.au/buy-a-car/all-new...,2006,Mercedes-Benz,"[C200, Kompressor, ]",WA,Maylands,USED Dealer ad,2021/10/08,0
1,616017a57591e422f04df82f,Great Deal Auto,2009 Ford Ranger XL,195565,Automatic,15500.0,We offer onsite finance through Australias lea...,https://www.carsguide.com.au/buy-a-car/all-new...,2009,Ford,"[Ranger, XL]",WA,Malaga,USED Dealer ad,2021/10/08,0
2,616017a57591e422f04df830,Fabcar,2004 Toyota Estima Aeras,84973,Automatic,29888.0,** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY ...,https://www.carsguide.com.au/buy-a-car/all-new...,2004,Toyota,"[Estima, Aeras]",WA,Perth,USED Dealer ad,2021/10/08,0
3,616017a57591e422f04df831,Osborne Park Nissan used,2018 Dodge N/A Laramie Crew Cab SWB,106000,Automatic,149999.0,THOUSANDS SPEND ON UPGRADES!! heavy duty rear ...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,Dodge,"[N/A, Laramie, Crew, Cab, SWB]",WA,Osborne Park,USED Dealer ad,2021/10/08,0
4,616017a57591e422f04df832,Subaru Wangara used,2015 Volkswagen Golf R,102001,Manual,39998.0,LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefull...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Volkswagen,"[Golf, R]",WA,Wangara,USED Dealer ad,2021/10/08,0


In [ ]:
# Checking for anomolies and outliers
df.sort_values(by=['Price'])

In [54]:
import numpy as np
df2 = df.groupby(['Suburb']).agg({'Price':[np.mean, np.median]})
df2.Price.mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of                       mean   median
Suburb                             
Abbotsford    24499.000000  24499.0
Aberglasslyn  28000.000000  28000.0
Acacia Ridge  31902.796610  31980.0
Acland         5490.000000   5490.0
Adamstown     13021.250000  13490.0
...                    ...      ...
Yundi         14500.000000  14500.0
Zetland       50032.010101  39990.0
caringbah     38143.846154  39990.0
concord       11152.846154  10999.0
sylvania      68022.491803  67988.0

[1914 rows x 2 columns]>

In [40]:
#  Create some statistics variables
df_suburb_grouped = df.loc[:,['Suburb','State Abrev', 'Price','KMs', 'Year']].set_index('Suburb')
df_suburb_grouped = df.groupby(['Suburb','State Abrev'])

# Mean price
mean_price = df['Price'].mean().round()

# Median price
median_price = df_suburb_grouped['Price'].median().round()

# Count of listings per suburb
count_listings_suburb = df['Suburb'].value_counts()

df_suburb_grouped.head()


,_id,Dealer Name,Year-make-model,KMs,Tansmission,Price,Comments,Source URL,Year,Make,Model,State Abrev,Suburb,Listing Type,Date scraped,Page Number
0,616017a57591e422f04df82e,Parkside Motor Co,2006 Mercedes-Benz C200 Kompressor,189474,Automatic,8990.0,Just arrived stunning C200 Kompressor Sport in...,https://www.carsguide.com.au/buy-a-car/all-new...,2006,Mercedes-Benz,"[C200, Kompressor, ]",WA,Maylands,USED Dealer ad,2021/10/08,0
1,616017a57591e422f04df82f,Great Deal Auto,2009 Ford Ranger XL,195565,Automatic,15500.0,We offer onsite finance through Australias lea...,https://www.carsguide.com.au/buy-a-car/all-new...,2009,Ford,"[Ranger, XL]",WA,Malaga,USED Dealer ad,2021/10/08,0
2,616017a57591e422f04df830,Fabcar,2004 Toyota Estima Aeras,84973,Automatic,29888.0,** VIEWINGS BY APPOINTMENT ONLY **** MOBILITY ...,https://www.carsguide.com.au/buy-a-car/all-new...,2004,Toyota,"[Estima, Aeras]",WA,Perth,USED Dealer ad,2021/10/08,0
3,616017a57591e422f04df831,Osborne Park Nissan used,2018 Dodge N/A Laramie Crew Cab SWB,106000,Automatic,149999.0,THOUSANDS SPEND ON UPGRADES!! heavy duty rear ...,https://www.carsguide.com.au/buy-a-car/all-new...,2018,Dodge,"[N/A, Laramie, Crew, Cab, SWB]",WA,Osborne Park,USED Dealer ad,2021/10/08,0
4,616017a57591e422f04df832,Subaru Wangara used,2015 Volkswagen Golf R,102001,Manual,39998.0,LIVE THE FULL HOT HATCH EXPERIENCE.. Tastefull...,https://www.carsguide.com.au/buy-a-car/all-new...,2015,Volkswagen,"[Golf, R]",WA,Wangara,USED Dealer ad,2021/10/08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47705,616068087591e422f04eb287,,2017 Land Rover Discovery TD4 SE,78500,Automatic,52500.0,This car is in great condition and has been an...,https://www.carsguide.com.au/buy-a-car/all-new...,2017,Land,"[Rover, Discovery, TD4, SE]",QLD,Dicky Beach,USED Private ad,2021/10/08,5338
47718,616068097591e422f04eb294,,2017 Land Rover Discovery TD4 SE,78500,Automatic,52500.0,This car is in great condition and has been an...,https://www.carsguide.com.au/buy-a-car/all-new...,2017,Land,"[Rover, Discovery, TD4, SE]",QLD,Dicky Beach,USED Private ad,2021/10/08,5339
47796,6160682d7591e422f04eb2e2,,2021 Suzuki Jimny GLX (qld),7500,Manual,39000.0,Selling my new 2021 Suzuki Jimny Manual GLX! S...,https://www.carsguide.com.au/buy-a-car/all-new...,2021,Suzuki,"[Jimny, GLX, (qld)]",QLD,Coolangatta,USED Private ad,2021/10/08,5347
47817,6160683b7591e422f04eb2f7,,2016 Volkswagen Polo 81 TSI Comfortline,7579,Automatic,18000.0,"This car is like new, very few kilometers, sel...",https://www.carsguide.com.au/buy-a-car/all-new...,2016,Volkswagen,"[Polo, 81, TSI, Comfortline]",QLD,Ninderry,USED Private ad,2021/10/08,5350


In [ ]:
df_suburb_grouped.to_csv("suburb_listings.csv", index=False, header=True)

In [34]:
mydf = pd.DataFrame(list(zip(mean_price, median_price)), columns = ['Mean Price', 'Median Price'])
mydf.head()

,Mean Price,Median Price
0,24499.0,24499.0
1,28000.0,28000.0
2,31903.0,31980.0
3,5490.0,5490.0
4,13021.0,13490.0


In [35]:
# Work out average Price by Suburb
# pd.options.display.float_format = '${:,.2f}'.format
df_state_grouped = df.loc[:,['State Abrev', 'Suburb', 'Price','KMs', 'Comments']]
df_state_grouped = df_state_grouped.groupby(['State Abrev','Suburb'])
df_state_grouped['Price'].mean().round()

State Abrev  Suburb   
ACT          Amaroo       56600.0
             Belconnen    29641.0
             Bonython     36500.0
             Braddon      11100.0
             Bruce        10000.0
                           ...   
WA           Winthrop     32000.0
             Woodvale     19833.0
             Yanchep      11500.0
             Yangebup     11429.0
             Yokine       24400.0
Name: Price, Length: 1988, dtype: float64